In [72]:
import pandas as pd 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier


In [40]:
## In the event that nltk.download("stopwords") is unavailable due to failed SSH verification, use this array 
## It contains ALL the stopwords that nltk has, and is also able to be amended at will for future use cases
custom_stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [41]:
df=pd.read_csv("data/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


## Data Cleaning 
In this portion, we will check to see if what data values are required, and we will also check if there are any data values that are null. If there are, we should fill them in with an empty space. 

Next, we will drop the ID, title and the author columns as they are not needed. 

Finally, we will clean the data by removing words with little semantic value and reduce all words to their lowercase and base form (e.g. "running" to "run") for easier processing.

## Clearing Null Values

In [43]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [44]:
df = df.fillna('')
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## Removing Uneccessary Columns

In [45]:
df = df.drop(['id', 'title', 'author'], axis = 1)
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


##

In [46]:
port_stem = PorterStemmer()

In [47]:
def stemming(content):
    new_content = re.sub('[^a-zA-Z]', ' ', content)
    new_content = new_content.lower()
    new_content = new_content.split()
    new_content = [port_stem.stem(x) for x in new_content if not x in custom_stopwords]
    new_content = ' '.join(new_content)
    return new_content

In [48]:
## Stem all text from the dataset
df['text'] = df['text'].apply(stemming)

## Separate data into dependant and independant features

In [67]:
x = df['text']
y = df['label']

## Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [68]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

# Creating the Model

In [65]:
model = DecisionTreeClassifier()

In [71]:
model.fit(x_train, y_train)
prediction = model.predict(x_test)
model.score(x_test, y_test)

0.885576923076923

## Saving the Model

In [ ]:
pickle.dump(vectorizer, open('vector.pkl', 'wb'))
pickle.dump(model, open('model.pkl', 'wb'))

In [76]:
load_vector = pickle.load(open('vector.pkl', 'rb'))
load_model = pickle.load(open('model.pkl', 'rb'))

In [84]:
def detector(news): 
    news = stemming(news)
    input = [news]
    new_load_vector = load_vector.transform(input)
    result = load_model.predict(new_load_vector)
    return result

In [98]:
result = detector("I won the 2016 election")
if result == [0]:
    print('Reliable')
else:
    print('Unreliable')

Unreliable
